/* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
you might need to do some digging, aand revise the Working with Relational Databases in Python chapter in the previous resource.

Otherwise, the questions in the case study are exactly the same as with Tier 1. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */


/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT name
  FROM Facilities
 WHERE membercost > 0;
 
Returns:
name
Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court

/* Q2: How many facilities do not charge a fee to members? */

SELECT COUNT(facid) AS count_free_to_members
  FROM Facilities
 WHERE membercost > 0;
 
Returns:
count_free_to_members
5

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

SELECT facid, name, membercost, monthlymaintenance
  FROM Facilities
 WHERE (membercost > 0)
       AND (membercost < 0.2 * monthlymaintenance);
       

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT *
  FROM Facilities
 WHERE facid IN (1, 5);

/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT name, monthlymaintenance, expense_label
  FROM Facilities;

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT firstname, surname, joindate
  FROM Members AS m,
       (SELECT MAX(joindate) AS last_date
       FROM Members) AS ld
 WHERE joindate = last_date;
 
Returns:
firstname surname joindate
Darren Smith 2012-09-26 18:08:45

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT DISTINCT f.name AS court_name, CONCAT(m.firstname, ' ', m.surname) AS member_name
  FROM Bookings AS b
       INNER JOIN Facilities AS f
             USING (facid)
       INNER JOIN Members AS m
             USING (memid)
 WHERE name LIKE 'Tennis Court%'
 ORDER BY member_name, court_name;

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT name as facility_name, CONCAT(firstname, ' ', surname) AS member_name, 
       CASE WHEN (b.memid = 0) THEN guestcost * slots
            ELSE membercost * slots
            END AS cost
  FROM Bookings AS b
       INNER JOIN Facilities AS f
             USING (facid)
       INNER JOIN Members AS m
             USING (memid)
 WHERE starttime LIKE '2012-09-14%'
HAVING cost > 30
 ORDER BY cost DESC;

/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT name as facility_name, CONCAT(firstname, ' ', surname) AS member_name, 
       CASE WHEN (b.memid = 0) THEN guestcost * slots
            ELSE membercost * slots
            END AS cost
  FROM (SELECT *
          FROM Bookings
         WHERE starttime LIKE '2012-09-14%') AS b
       INNER JOIN Facilities AS f
             USING (facid)
       INNER JOIN Members AS m
             USING (memid)
HAVING cost > 30
 ORDER BY cost DESC;


/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

In [1]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


Write my own query function that returns a pandas DataFrame:

In [2]:
import pandas as pd

def dataframe_from_query_results(query):
    """
    Run the query and print the results.
    :param query: a SQL query string
    :return:
    """
    
    database = "sqlite_db_pythonsqlite.db"
    # create a database connection
    conn = create_connection(database)
    #conn.row_factory = sqlite3.Row
    
    with conn:
        #cur = conn.cursor()
        #cur.execute(query)
        cur = conn.execute(query)
        
        rows = cur.fetchall()
        
        names = [desc[0] for desc in cur.description]
        #print(names)
        
        df = pd.DataFrame(rows, columns = names)
        
        df
            
    return df

dataframe_from_query_results("""SELECT * FROM Facilities;""")

2.6.0


,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


QUESTIONS:

/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [3]:
query = """
SELECT name, SUM(cost) AS revenue
FROM (SELECT name, 
             CASE WHEN (b.memid = 0) THEN guestcost * slots
                  ELSE membercost * slots
                  END AS cost
        FROM Bookings AS b
             INNER JOIN Facilities AS f
                   USING (facid)
             INNER JOIN Members AS m
                   USING (memid)
       ) as c
GROUP BY name
HAVING revenue < 1000
ORDER BY revenue;
"""

dataframe_from_query_results(query)

2.6.0


,name,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [4]:
query = """
SELECT m1.surname AS surname_member, m1.firstname AS firstname_member, m2.surname AS surname_recommender, m2.firstname AS firstname_recommender
FROM Members AS m1
     LEFT JOIN Members AS m2
          ON m1.recommendedby = m2.memid
WHERE m1.surname != 'GUEST'
ORDER BY surname_member, firstname_member;
"""

df = dataframe_from_query_results(query)
df

2.6.0


,surname_member,firstname_member,surname_recommender,firstname_recommender
0,Bader,Florence,Stibbons,Ponder
1,Baker,Anne,Stibbons,Ponder
2,Baker,Timothy,Farrell,Jemima
3,Boothe,Tim,Rownam,Tim
4,Butters,Gerald,Smith,Darren
5,Coplin,Joan,Baker,Timothy
6,Crumpet,Erica,Smith,Tracy
7,Dare,Nancy,Joplette,Janice
8,Farrell,David,None,None
9,Farrell,Jemima,None,None


/* Q12: Find the facilities with their usage by member, but not guests */

The instructions are vague. If they mean "total usage by any member, but not guests", then here is the query:

In [5]:
query = """
SELECT name AS facility_name, COUNT(*) AS usage
FROM Bookings AS b
     INNER JOIN Facilities AS f
           USING (facid)
     INNER JOIN Members AS m
           USING (memid)
WHERE surname != 'GUEST'
GROUP BY facility_name
ORDER BY facility_name
;
"""

df = dataframe_from_query_results(query)
df

2.6.0


,facility_name,usage
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


If they are asking for the usage aggregated by each member, here is the query:

In [6]:
query = """
SELECT name AS facility_name, surname, firstname, COUNT(*) AS usage
FROM Bookings AS b
     INNER JOIN Facilities AS f
           USING (facid)
     INNER JOIN Members AS m
           USING (memid)
WHERE surname != 'GUEST'
GROUP BY facility_name, surname, firstname
ORDER BY facility_name, usage DESC
;
"""

df = dataframe_from_query_results(query)
df

2.6.0


,facility_name,surname,firstname,usage
0,Badminton Court,Smith,Darren,132
1,Badminton Court,Smith,Tracy,32
2,Badminton Court,Mackenzie,Anna,30
3,Badminton Court,Butters,Gerald,20
4,Badminton Court,Stibbons,Ponder,16
...,...,...,...,...
197,Tennis Court 2,Farrell,David,1
198,Tennis Court 2,Farrell,Jemima,1
199,Tennis Court 2,Purview,Millicent,1
200,Tennis Court 2,Rumney,Henrietta,1


/* Q13: Find the facilities usage by month, but not guests */

Again, this question is vague. I am assuming:

- Aggregating usage by each facility, as in the previous question, rather than total usage of all facilities.

- Aggregating usage by YYYY-MM, rather than MM. There are only 3 months in the data, so this shouldn't matter anyway.

In [7]:
query = """
SELECT name AS facility_name, SUBSTR(starttime, 1, 7) AS month, COUNT(*) AS usage
FROM Bookings AS b
     INNER JOIN Facilities AS f
           USING (facid)
WHERE memid != 0
GROUP BY facility_name, month
ORDER BY facility_name, usage
;
"""

df = dataframe_from_query_results(query)
df

2.6.0


,facility_name,month,usage
0,Badminton Court,2012-07,51
1,Badminton Court,2012-08,132
2,Badminton Court,2012-09,161
3,Massage Room 1,2012-07,77
4,Massage Room 1,2012-08,153
5,Massage Room 1,2012-09,191
6,Massage Room 2,2012-07,4
7,Massage Room 2,2012-08,9
8,Massage Room 2,2012-09,14
9,Pool Table,2012-07,103


If you want the monthly usage of all facilities added together, here it is:

In [8]:
query = """
SELECT SUBSTR(starttime, 1, 7) AS month, COUNT(*) AS usage
FROM Bookings AS b
     INNER JOIN Facilities AS f
           USING (facid)
WHERE memid != 0
GROUP BY month
ORDER BY usage
;
"""

df = dataframe_from_query_results(query)
df

2.6.0


,month,usage
0,2012-07,480
1,2012-08,1168
2,2012-09,1512
